# References

## References to Values
As an example, let’s suppose we’re going to build a table of murderous Renaissance artists and the works they’re known for. Rust’s standard library includes a hash table type, so we can define our type like this:

In [3]:
use std::collections::HashMap;
type Table = HashMap<String, Vec<String>>;

In other words, this is a hash table that maps `String` values to `Vec<String> values`, taking the name of an artist to a list of the names of their works. You can iterate over the entries of a `HashMap` with a for loop, so we can write a function to print out a Table:

In [4]:
fn show(table: Table) {
    for (artist, works) in table {
        println!("works by {}:", artist); 
        for work in works {
            println!("  {}", work);
        }
    } 
}

In [10]:
let mut table: Table = Table::new(); 
table.insert("Gesualdo".to_string(),
             vec!["many madrigals".to_string(),
                  "Tenebrae Responsoria".to_string()]);
table.insert("Caravaggio".to_string(),
             vec!["The Musicians".to_string(),
                  "The Calling of St. Matthew".to_string()]);
table.insert("Cellini".to_string(),
             vec!["Perseus with the head of Medusa".to_string(),
                 "a salt cellar".to_string()]);
show(table);

works by Cellini:
  Perseus with the head of Medusa
  a salt cellar
works by Gesualdo:
  many madrigals
  Tenebrae Responsoria
works by Caravaggio:
  The Musicians
  The Calling of St. Matthew


In fact, if we look into the definition of show, the outer for loop takes ownership of the hash table and consumes it entirely; and the inner for loop does the same to each of the vectors.

The right way to handle this is to use references. A reference lets you access a value without affecting its ownership. References come in two kinds:
* A shared reference lets you read but not modify its referent. However, you can have as many shared references to a particular value at a time as you like. The expression &e yields a shared reference to e’s value; if e has the type `T`, then `&e` has the type `&T`, pronounced “ref T.” **Shared references are `Copy`**.
* If you have a mutable reference to a value, you may both read and modify the value. However, you may not have any other references of any sort to that value active at the same time. The expression &mut e yields a mutable reference to e’s value; you write its type as `&mut T`, which is pronounced “ref mute T.” **Mutable references are not `Copy`**.

You can think of the distinction between shared and mutable references as a way to enforce a multiple readers or single writer rule at compile time. In fact, this rule doesn’t apply only to references; it covers the borrowed value’s owner as well. As long as there are shared references to a value, not even its owner can modify it; the value is locked down. Nobody can modify table while show is working with it. Similarly, if there is a mutable reference to a value, it has exclusive access to the value; you can’t use the owner at all, until the mutable reference goes away. Keeping sharing and mutation fully separate turns out to be essential to memory safety.

In [14]:
fn rshow(table: &Table) {
    for (artist, works) in table {
        println!("works by {}:", artist); 
        for work in works {
            println!("  {}", work);
        }
    } 
}

The type of show’s parameter table has changed from Table to `&Table`: instead of passing the table by value (and hence moving ownership into the function), we’re now passing a shared reference. That’s the only textual change. But how does this play out as we work through the body?

Whereas our original outer for loop took ownership of the HashMap and consumed it, in our new version it receives a shared reference to the HashMap. Iterating over a shared reference to a HashMap is defined to produce shared references to each entry’s key and value: artist has changed from a String to a `&String`, and works from a `Vec<String>` to a `&Vec<String>`.

The inner loop is changed similarly. Iterating over a shared reference to a vector is defined to produce shared references to its elements, so work is now a `&String`. No ownership changes hands anywhere in this function; it’s just passing around non-owning references.

Now, if we wanted to write a function to alphabetize the works of each artist, a shared reference doesn’t suffice, since shared references don’t permit modification. Instead, the sorting function needs to take a mutable reference to the table:


In [15]:
fn sort_works(table: &mut Table) { 
    for (_artist, works) in table {
            works.sort();
        }
}

This mutable borrow grants sort_works the ability to read and modify our structure,
as required by the vectors’ sort method.

### Rust References Versus C++ References
If you’re familiar with references in C++, they do have something in common with Rust references. Most importantly, they’re both just addresses at the machine level. But in practice, Rust’s references have a very different feel.

In C++, references are created implicitly by conversion, and dereferenced implicitly too:

In [19]:
// C++ code!
// int x=10;
// int &r = x; // initialization creates reference implicitly
// assert(r == 10); // implicitly dereference r to see x's value
// r = 20; // stores 20 in x, r itself still points to x

In Rust, references are created explicitly with the & operator, and dereferenced explicitly with the * operator:

In [23]:
// Back to Rust code from this point onward.
let x =10;
{
    let r = &x; // &x is a shared reference to x 
    assert!(*r == 10); // explicitly dereference r
};

In [26]:
let mut y = 32; 
{
    let m = &mut y; // &mut y is a mutable reference to y
    *m += 32; // explicitly dereference m to set y's value
    assert!(*m == 64); // and to see y's new value
};

But you might recall that, when we fixed the show function to take the table of artists by reference instead of by value, we never had to use the `*` operator. Why is that?

Since references are so widely used in Rust, the `.` operator implicitly dereferences its left operand, if needed:

In [32]:
struct Anime { name: &'static str, bechdel_pass: bool }
let aria = Anime { name: "Aria: The Animation", bechdel_pass: true }; 
{
    let anime_ref = &aria;
    assert_eq!(anime_ref.name, "Aria: The Animation");
    // Equivalent to the above, but with the dereference written out:
    assert_eq!((*anime_ref).name, "Aria: The Animation");
};

The `.` operator can also implicitly borrow a reference to its left operand, if needed for a method call. For example, Vec’s sort method takes a mutable reference to the vector, so these two calls are equivalent:

In [34]:
let mut v: Vec<i32> = vec![1973, 1968];
v.sort(); // implicitly borrows a mutable reference to v 
(&mut v).sort(); // equivalent, but more verbose

In a nutshell, whereas C++ converts implicitly between references and lvalues (that is, expressions referring to locations in memory), with these conversions appearing any‐ where they’re needed, in Rust you use the `&` and `*` operators to create and follow references, with the exception of the . operator, which borrows and dereferences implicitly.

### Assigning References
Assigning a reference to a variable makes that variable point somewhere new:

In [40]:
let x=10; 
let y=20;
{
    let mut r=&x;
    let b = true;
    if b{r=&y;}
    assert!(*r == 10 || *r == 20);
};

### References to References

In [43]:
struct Point { x: i32, y: i32 }
let point = Point { x: 1000, y: 729 }; 
{
    let r: &Point = &point;
    let rr: &&Point = &r;
    let rrr: &&&Point = &rr;
    assert_eq!(rrr.y, 729);
};

![image.png](asset/ch5/1.png)

So the expression `rrr.y`, guided by the type of `rrr`, actually traverses three references to get to the `Point` before fetching its `y` field.

### Comparing References

In [48]:
let x=10; 
let y=10;
{
    let rx=&x; 
    let ry=&y;
    let rrx = &rx; 
    let rry = &ry;
    assert!(rrx <= rry);
    assert!(rrx == rry);

    assert!(rx == ry); // their referents are equal 
    assert!(!std::ptr::eq(rx, ry)); // but occupy different addresses
};

In [49]:
let x=10; 
let y=10;
{
    let rx=&x; 
    let ry=&y;
    let rrx = &rx; 
    let rry = &ry;
    
    assert!(rx == rrx); // error: type mismatch: `&i32` vs `&&i32` 
    assert!(rx == *rrx); // this is okay
};

Error: can't compare `{integer}` with `&{integer}`

### References Are Never Null
Rust references are never null. There’s no analogue to C’s NULL or C++’s nullptr. There is no default initial value for a reference (you can’t use any variable until it’s been initialized, regardless of its type) and Rust won’t convert integers to references (outside of unsafe code), so you can’t convert zero into a reference.

C and C++ code often uses a null pointer to indicate the absence of a value: for exam‐ ple, the malloc function returns either a pointer to a new block of memory or nullptr if there isn’t enough memory available to satisfy the request. In Rust, if you need a value that is either a reference to something or not, use the type `Option<&T>`. At the machine level, Rust represents None as a null pointer and `Some(r)`, where `r` is a `&T` value, as the nonzero address, so `Option<&T>` is just as efficient as a nullable pointer in C or C++, even though it’s safer: its type requires you to check whether it’s None before you can use it.

### Borrowing References to Arbitrary Expressions
Whereas C and C++ only let you apply the & operator to certain kinds of expressions, Rust lets you borrow a reference to the value of any sort of expression at all:

In [52]:
fn factorial(n: usize) -> usize { 
    (1..n+1).product()
}
{
    let r = &factorial(6); // Arithmetic operators can see through one level of references. 
    assert_eq!(r + &1009, 1729);
};

In situations like this, Rust simply creates an anonymous variable to hold the expres‐ sion’s value and makes the reference point to that. The lifetime of this anonymous variable depends on what you do with the reference:
* If you immediately assign the reference to a variable in a let statement (or make it part of some struct or array that is being immediately assigned), then Rust makes the anonymous variable live as long as the variable the let initializes. In the preceding example, Rust would do this for the referent of r.
* Otherwise, the anonymous variable lives to the end of the enclosing statement. In our example, the anonymous variable created to hold 1009 lasts only to the end of the assert_eq! statement.

## Reference Safety
### Receiving References as Function Arguments

In [53]:
 // This code has several problems, and doesn't compile.
static mut STASH: &i32; 
fn f(p:&i32){ 
    STASH=p;
}

Error: free static item without body

Rust’s equivalent of a global variable is called a static: it’s a value that’s created when the program starts and lasts until it terminates.
* Every static must be initialized.
* Mutable statics are inherently not thread-safe (after all, any thread can access a static at any time), and even in single-threaded programs, they can fall prey to other sorts of reentrancy problems. For these reasons, you may access a mutable static only within an unsafe block. In this example we’re not concerned with those particular problems, so we’ll just throw in an unsafe block and move on.

With those revisions made, we now have the following:

In [54]:
static mut STASH: &i32 = &128;
fn f(p: &i32) { // still not good enough
    unsafe { 
        STASH = p;
    } 
}

Error: lifetime may not live long enough

We’re almost done. To see the remaining problem, we need to write out a few things that Rust is helpfully letting us omit. The signature of `f` as written here is actually shorthand for the following:
```Rust
fn f<'a>(p: &'a i32) { ... }
```
Here, the lifetime 'a (pronounced “tick A”) is a lifetime parameter of `f`. You can read `<'a>` as “for any lifetime 'a” so when we write `fn f<'a>(p: &'a i32)`, we’re defining a function that takes a reference to an `i32` with any given lifetime `'a`.
Since we must allow `'a` to be any lifetime, things had better work out if it’s the smallest possible lifetime: one just enclosing the call to `f`. This assignment then becomes a point of contention:
```Rust
STASH = p;
```
Since `STASH` lives for the program’s entire execution, the reference type it holds must have a lifetime of the same length; Rust calls this the `'static` lifetime. But the lifetime of p’s reference is some `'a`, which could be anything, as long as it encloses the call to f.

In [55]:
static mut STASH: &i32 = &10;
fn f(p: &'static i32) { 
    unsafe {
        STASH = p; 
    }
}

### Passing References to Functions
```Rust
fn f(p: &'static i32) { ... } 
let x = 10;
f(&x);
```
This fails to compile: the reference `&x` must not outlive `x`, but by passing it to `f`, we constrain it to live at least as long as `'static`. There’s no way to satisfy everyone here, so Rust rejects the code.

### Returning References



In [57]:
// v should have at least one element.
fn smallest(v: &[i32]) -> &i32 { 
    let mut s = &v[0];
    for r in &v[1..] {
        if*r<*s{s=r;} 
    }
    s
}

let s; {
    let parabola = [9, 4, 1, 0, 1, 4, 9]; 
    s = smallest(&parabola);
}
assert_eq!(*s, 0); // bad: points to element of dropped array

Error: `parabola` does not live long enough

From smallest’s signature, we can see that its argument and return value must have the same lifetime, 'a. In our call, the argument &parabola must not outlive parabola itself, yet smallest’s return value must live at least as long as s. There’s no possible lifetime 'a that can satisfy both constraints

In [59]:
{
    let parabola = [9, 4, 1, 0, 1, 4, 9];
    let s = smallest(&parabola);
    assert_eq!(*s, 0); // fine: parabola still alive
};

### Structs Containing References
Whenever a reference type appears inside another type’s definition, you must write out its lifetime.
```Rust
struct S {
    r: &'static i32
}
```
This says that r can only refer to i32 values that will last for the lifetime of the pro‐ gram, which is rather limiting. The alternative is to give the type a lifetime parameter 'a and use that for r:
```Rust
struct S<'a> { 
    r: &'a i32
}
```
Now the S type has a lifetime, just as reference types do. Each value you create of type S gets a fresh lifetime 'a, which becomes constrained by how you use the value. The lifetime of any reference you store in r had better enclose 'a, and 'a must outlast the lifetime of wherever you store the S.

How does a type with a lifetime parameter behave when placed inside some other type?
```Rust
struct D {
    s: S // not adequate
}
```
Rust is skeptical, just as it was when we tried placing a reference in S without specify‐ ing its lifetime:

We could give s the 'static lifetime. This works:
```Rust
struct D {
    s: S<'static>
}
```
With this definition, the `s` field may only borrow values that live for the entire execution of the program. That’s somewhat restrictive, but it does mean that `D` can’t possibly borrow a local variable; there are no special constraints on `D`’s lifetime.

The error message from Rust actually suggests another approach, which is more general:
```text
help: consider introducing a named lifetime parameter
  |
7 | struct D<'a> {
8 |     s: S<'a>
  |
```
Here, we give D its own lifetime parameter and pass that to S:
```Rust
struct D<'a> { 
    s: S<'a>
}
```
By taking a lifetime parameter `'a` and using it in s’s type, we’ve allowed Rust to relate D value’s lifetime to that of the reference its `S` holds.

We showed earlier how a function’s signature exposes what it does with the references we pass it. Now we’ve shown something similar about types: a type’s lifetime parameters always reveal whether it contains references with interesting (that is, non-'static) lifetimes and what those lifetimes can be.

For example, suppose we have a parsing function that takes a slice of bytes and returns a structure holding the results of the parse:
```Rust
fn parse_record<'i>(input: &'i [u8]) -> Record<'i> { ... }
```
Without looking into the definition of the Record type at all, we can tell that, if we receive a Record from `parse_record`, whatever references it contains must point into the input buffer we passed in, and nowhere else (except perhaps at `'static` values).

It’s not just references and types like S that have lifetimes. Every type in Rust has a lifetime, including `i32` and String. Most are simply `'static`, meaning that values of those types can live for as long as you like; for example, a `Vec<i32>` is self-contained and needn’t be dropped before any particular variable goes out of scope. But a type like `Vec<&'a i32>` has a lifetime that must be enclosed by `'a`: it must be dropped while its referents are still alive.

### Distinct Lifetime Parameters
Suppose you’ve defined a structure containing two references like this:

Both references use the same lifetime 'a. This could be a problem if your code wants to do something like this:


In [66]:
struct S<'a> { 
    x: &'a i32,
    y: &'a i32 
}

In [67]:
let x = 10; 
{
    let r;
    {
        let y = 20; 
        {
            let s = S {x:&x,y:&y};
            r = s.x; 
        }
    }
    println!("{}", r);
};

Error: `y` does not live long enough

In [68]:
struct S<'a, 'b> { 
    x: &'a i32,
    y: &'b i32 
}

In [69]:
let x = 10; 
{
    let r;
    {
        let y = 20; 
        {
            let s = S {x:&x,y:&y};
            r = s.x; 
        }
    }
    println!("{}", r);
};

10


Function signatures can have similar effects. Suppose we have a function like this:
```Rust
fn f<'a>(r: &'a i32, s: &'a i32) -> &'a i32 { r } // perhaps too tight
```
Here, both reference parameters use the same lifetime `'a`, which can unnecessarily constrain the caller in the same way we’ve shown previously. If this is a problem, you can let parameters’ lifetimes vary independently:
```Rust
fn f<'a, 'b>(r: &'a i32, s: &'b i32) -> &'a i32 { r } // looser
```
The downside to this is that adding lifetimes can make types and function signatures harder to read. Your authors tend to try the simplest possible definition first and then loosen restrictions until the code compiles. Since Rust won’t permit the code to run unless it’s safe, simply waiting to be told when there’s a problem is a perfectly acceptable tactic.

### Omitting Lifetime Parameters
We’ve shown plenty of functions so far in this book that return references or take them as parameters, but we’ve usually not needed to spell out which lifetime is which. The lifetimes are there; Rust is just letting us omit them when it’s reasonably obvious what they should be.

In the simplest cases, you may never need to write out lifetimes for your parameters. Rust just assigns a distinct lifetime to each spot that needs one. For example:
```Rust
struct S<'a, 'b> { 
    x: &'a i32,
    y: &'b i32 
}
fn sum_r_xy(r: &i32, s: S) -> i32 { 
    r+s.x+s.y
}
```
This function’s signature is shorthand for:
```Rust
fn sum_r_xy<'a, 'b, 'c>(r: &'a i32, s: S<'b, 'c>) -> i32
```
If your function is a method on some type and takes its self parameter by reference, then that breaks the tie: Rust assumes that self’s lifetime is the one to give everything in your return value.

In [70]:
struct StringTable { 
    elements: Vec<String>,
}
impl StringTable {
    fn find_by_prefix(&self, prefix: &str) -> Option<&String> {
        for i in 0 .. self.elements.len() {
        if self.elements[i].starts_with(prefix) {
            return Some(&self.elements[i]); }
        }
        None
    } 
}

The find_by_prefix method’s signature is shorthand for:
```Rust
fn find_by_prefix<'a, 'b>(&'a self, prefix: &'b str) -> Option<&'a String>
```

## Sharing Versus Mutation


In [77]:
let v: Vec<i32> = vec![4, 8, 19, 27, 34, 10];
let r: &Vec<i32> =&v;
let aside: Vec<i32> = v; // move vector to aside
r[0]; // bad: uses `v`, which is now uninitialized

Error: The variable `r` contains a reference with a non-static lifetime so
can't be persisted. You can prevent this error by making sure that the
variable goes out of scope - i.e. wrapping the code in {}.

![image.png](asset/ch5/2.png)

In [78]:
let v: Vec<i32> = vec![4, 8, 19, 27, 34, 10]; 
{
    let r: &Vec<i32> = &v;
    r[0]; // ok: vector is still there 
}
let aside: Vec<i32> = v;

Suppose we have a handy function to extend a vector with the elements of a slice:

In [89]:
fn extend(vec: &mut Vec<f64>, slice: &[f64]) { 
    for elt in slice {
        vec.push(*elt);
    }
}
{
    let mut wave: Vec<f64> = Vec::new(); 
    let head = vec![0.0, 1.0]; 
    let tail: [f64; 2] = [0.0, -1.0];
    extend(&mut wave, &head); // extend wave with another vector
    extend(&mut wave, &tail); // extend wave with an array
    assert_eq!(wave, vec![0.0, 1.0, 0.0, -1.0]);

    // illagel, because read and write cannot exist at the same time
    //extend(&mut wave, &wave); 
};

Error: cannot borrow `wave` as immutable because it is also borrowed as mutable

![image.png](asset/ch5/3.png)

In [ ]:
let mut x = 10;
let r1 = &x;
let r2 = &x; // ok: multiple shared borrows permitted
x += 10; // error: cannot assign to `x` because it is borrowed 
let m = &mut x; // error: cannot borrow `x` as mutable because it is
                // also borrowed as immutable
println!("{}, {}, {}", r1, r2, m); // the references are used here,
                                       // so their lifetimes must last
                                       // at least this long
let mut y = 20;
let m1 = &mut y;
let m2 = &mut y; // error: cannot borrow as mutable more than once
let z = y; // error: cannot use `y` because it was mutably borrowed println!("{}, {}, {}", m1, m2, z); // references are used here

In [ ]:
let mut w = (107, 109); 
let r = &w;
let r0 = &r.0; // ok: reborrowing shared as shared
let m1 = &mut r.1; // error: can't reborrow shared as mutable
println!("{}", r0); // r0 gets used here

In [ ]:
let mut v = (136, 139); 
let m = &mut v; // ok: reborrowing mutable from mutable
let m0 = &mut m.0; 
*m0 = 137;  // ok: reborrowing shared from mutable,
            // and doesn't overlap with m0
let r1 = &m.1; 
v.1; // error: access through other paths still forbidden
println!("{}", r1); // r1 gets used here

Here’s another example of the kind of bug these rules catch. Consider the following C++ code, meant to manage a file descriptor. To keep things simple, we’re only going to show a constructor and a copying assignment operator, and we’re going to omit error handling:
```
struct File { 
    int descriptor;

    File(int d) : descriptor(d) { }
    File& operator=(const File &rhs) { 
        close(descriptor);
        descriptor = dup(rhs.descriptor); 
        return *this;
    } 
};
```
The assignment operator is simple enough, but fails badly in a situation like this:
```
File f(open("foo.txt", ...)); ...
f = f;
```
If we assign a File to itself, both rhs and *this are the same object, so operator= closes the very file descriptor it’s about to pass to dup. We destroy the same resource we were meant to copy.

In Rust, the analogous code would be:

In [ ]:
struct File { descriptor: i32
}
fn new_file(d: i32) -> File { 
    File { descriptor: d }
}
fn clone_from(this: &mut File, rhs: &File) { 
    close(this.descriptor);
    this.descriptor = dup(rhs.descriptor);
}

let mut f = new_file(open("foo.txt", ...)); ...
clone_from(&mut f, &f);  // eliminate the issue with the constraint--read & write conflict

![image.png](asset/ch5/4.png)

## Taking Arms Against a Sea of Objects
